In [ ]:
import fh
from fh import FH
from treeBuilder import TreeBuilder
import PydalChanel as pydal

lp = FH()
tempo = pydal.tempo

In [ ]:
tree = TreeBuilder([0]*8, fh.transformTranspositionVector, 4, lambda depth: 5)

In [ ]:
for c in tree.nodesByDepth():
    print c, len(c)

In [ ]:
getVec = lambda:tree.executeStepwise("+ \/*3 -")


In [ ]:
chan = 0
lp.startChannel(1, lambda buf: fh.vectorTranspose(buf, lp.roots[chan], lp.scales[chan], getVec()), lp.loops[chan][0])

In [ ]:
v = [0]*8
for i in range(10):
    v = fh.transformTranspositionVector(v)
print v